In [1]:
%pip install sqlalchemy
%pip install pymysql
%pip install cryptography
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Exhaustive Performance

In [19]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="r00t",
  port="3309",
    database="pippo"
)
mycursor = mydb.cursor()
TABLE = "pippo.mytable_gamma_bad"
FOLDER = "data"
MAX_NUMBER_OF_NODES = 7
MAX_NUMBER_OF_SERVICES = 7





In [9]:
result_dict = {}

mycursor.execute(f"""SELECT number_of_services, number_of_nodes, avg(execution_time) as execution
        FROM {TABLE}
        WHERE window_size = number_of_nodes
        GROUP BY number_of_nodes, number_of_services
        ORDER BY number_of_services
    """)

for row in mycursor:
    number_of_services = row[0]
    number_of_nodes = row[1]
    execution_time = row[2]

    if number_of_services not in result_dict:
        result_dict[number_of_services] = [execution_time]
    else:
        result_dict[number_of_services].append(execution_time)

# Print the formatted output
with open(f"{FOLDER}/exhaustive_performance_{TABLE}.dat", "w") as file:
    for number_of_nodes, execution_times in result_dict.items():
        starred = " ".join(map(str, execution_times))
        file.write(f"{number_of_nodes} {starred}\n")
        print(number_of_nodes, starred)




ProgrammingError: 1146 (42S02): Table 'pippo.mytable_gamma_bad' doesn't exist

# Window Time Performance 

In [6]:


mycursor = mydb.cursor()
#result_dict = {}
for n in range(2,8):

    mycursor.execute(f"""SELECT window_size,number_of_services, number_of_nodes, avg(execution_time) as execution
            FROM  {TABLE}
            WHERE number_of_nodes =  {n}
            GROUP BY number_of_nodes, number_of_services,window_size
            ORDER BY window_size

        """)


    result_dict = {}
    for row in mycursor:
        window_size, number_of_services, number_of_nodes, execution_time  = row
        if window_size not in result_dict:
            result_dict[window_size] = {number_of_services: [execution_time]}
        else:
            if number_of_services not in result_dict[window_size]:
                result_dict[window_size][number_of_services] = [execution_time]
            else:
                result_dict[window_size][number_of_services].append(execution_time)

    with open(f"{FOLDER}/window_time_performance_{TABLE}_n{n}.dat", "w") as file:
        for window_size, services in result_dict.items():
            for number_of_services, execution_times in services.items():
                starred = " ".join(map(str, execution_times))
                file.write(f"{number_of_services} {starred}\n")
                print(window_size, number_of_services, starred)

            file.write("\n\n")


    # Print the formatted output
    # with open(f"window_time_performance_{TABLE}_n{n}_w{w}.dat", "w") as file:
    #     for number_of_nodes, execution_times in result_dict.items():
    #         starred = " ".join(map(str, execution_times))
    #         file.write(f"{number_of_nodes} {starred}\n")
    #         print(number_of_nodes, starred)




1.0 2.0 347.29541611111114
1.0 3.0 463.35934988888886
1.0 4.0 599.2895064444444
1.0 5.0 682.9574964444444
1.0 6.0 803.1902174444444
2.0 2.0 392.6762254444445
2.0 3.0 801.1336196666666
2.0 4.0 1366.8615504444444
2.0 5.0 2092.789003
2.0 6.0 2954.082864444444
3.0 2.0 385.1331866666667
3.0 3.0 1191.2728139999997
3.0 4.0 2819.3528526666664
3.0 5.0 5476.306600777778
3.0 6.0 9359.66719888889
1.0 2.0 460.70542355555557
1.0 3.0 606.112430111111
1.0 4.0 767.9123361111111
1.0 5.0 902.8550632222223
1.0 6.0 1071.8157087777777
2.0 2.0 591.0031093333333
2.0 3.0 1188.9616012222223
2.0 4.0 2010.541966888889
2.0 5.0 3066.9897326666664
2.0 6.0 4446.676598222222
3.0 2.0 741.9226948888889
3.0 3.0 2346.9961632222216
3.0 4.0 5399.425157777778
3.0 5.0 10594.81294488889
3.0 6.0 18322.330477777778
4.0 2.0 748.2558973333332
4.0 3.0 3671.9400215555556
4.0 4.0 11271.009261222223
4.0 5.0 28084.670620222223
4.0 6.0 57917.33728611111
1.0 2.0 587.8818148888889
1.0 3.0 754.7655778888889
1.0 4.0 935.3254652222222
1.0 5.

# Window Quality Performance

In [22]:
mycursor = mydb.cursor()
#result_dict = {}
for n in range(3,MAX_NUMBER_OF_NODES+1):
    for w in range(1,n+1):
        query = f"""SELECT number_of_services, number_of_nodes, avg(metric), avg(percentage)
                FROM {TABLE}
                WHERE number_of_nodes = {n} AND window_size = {w}
                GROUP BY number_of_nodes, number_of_services,window_size
                ORDER BY number_of_services
            """
        mycursor.execute(query)

        result_dict = {}
        for row in mycursor:
            number_of_services = row[0]
            number_of_nodes = row[1]
            execution_time = row[2]
            percentage = row[3]

            if number_of_services not in result_dict:
                result_dict[number_of_services] = [execution_time, percentage]
            else:
                result_dict[number_of_services].append(execution_time, percentage)

        # Print the formatted output
        with open(f"{FOLDER}/window_quality_performance_{TABLE}_n{n}_w{w}.dat", "w") as file:
            for number_of_nodes, execution_times in result_dict.items():
                starred = " ".join(map(str, execution_times))
                file.write(f"{number_of_nodes} {starred}\n")
                print(number_of_nodes, starred, percentage)

2.0 0.10400758319144843 0.33101963040110555 0.41845376937981543
3.0 0.09308261398998094 0.380981647904261 0.41845376937981543
4.0 0.08476862149987995 0.42233394495433313 0.41845376937981543
5.0 0.0830902544780983 0.4263918986238528 0.41845376937981543
6.0 0.08586023943857832 0.41845376937981543 0.41845376937981543
2.0 0.10226777366250572 0.3356656410456024 0.4377762838906199
3.0 0.09103205573806572 0.3879421177455862 0.4377762838906199
4.0 0.0849966899321996 0.4197737262321501 0.4377762838906199
5.0 0.08012489198827216 0.4440325075328115 0.4377762838906199
6.0 0.08227585491788664 0.4377762838906199 0.4377762838906199
2.0 0.10158829930327326 0.33976815942363875 0.43917400409203206
3.0 0.08872680954668034 0.3994000795530219 0.43917400409203206
4.0 0.0839728225936227 0.42539744195542123 0.43917400409203206
5.0 0.08012489198827216 0.4440325075328115 0.43917400409203206
6.0 0.08213379415702285 0.43917400409203206 0.43917400409203206
2.0 0.11679934597688993 0.2724859488557343 0.3257561724165

In [8]:


mycursor = mydb.cursor()
#result_dict = {}
for n in range(2,6):
    for w in range(1,n):
        query = f"""SELECT number_of_services, number_of_nodes, avg(percentage)
                FROM pippo.mytable
                WHERE number_of_nodes = {n} AND window_size = {w}
                GROUP BY number_of_nodes, number_of_services,window_size
                ORDER BY number_of_services
            """
        mycursor.execute(query)

        result_dict = {}
        for row in mycursor:
            number_of_services = row[0]
            number_of_nodes = row[1]
            execution_time = row[2]

            if number_of_services not in result_dict:
                result_dict[number_of_services] = [execution_time]
            else:
                result_dict[number_of_services].append(execution_time)

        # Print the formatted output
        with open(f"{FOLDER}/window_percentage_performance_n{n}_w{w}.dat", "w") as file:
            for number_of_nodes, execution_times in result_dict.items():
                starred = " ".join(map(str, execution_times))
                file.write(f"{wino} {number_of_nodes} {starred}\n")
                print(number_of_nodes, starred)


NameError: name 'wino' is not defined

In [ ]:
mydb.close()